## Spark SQL

Spark SQL是架在Spark core之上允许我们用SQL去操作大型数据库的高级层。

首先你需要了解的是，**我们需要在之前的`SparkContext`基础上架上一层`SQLContext`，然后才能完成后续的SQL。**<br>
如下：

In [1]:
from pyspark import SQLContext, Row
sqlCtx = SQLContext(sc)

ImportError: No module named pyspark

Spark SQL里有一种新的RDD，叫做**`SchemaRDD`**，其实你可以把他看作一个table。<br>
schema RDD类似一种图表的RDD(想象成Excel那样)，每个item都是一个`Row`

最简单的初始化`SchemaRDD`方法是先用`sc.parallelize`产出一个RDD，如下：

In [2]:
rdd = sc.parallelize([
        Row(name='John', home='Brussels', age=35),
        Row(name='Jack', home='Brussels', age=32),
        Row(name='Jane', home='Leuven', age=42),
        Row(name='Jill', home='Mechelen', age=53),
    ])

然后通过下面的方式把RDD转成一个SchemaRDD：

In [3]:
schemaRDD = sqlCtx.inferSchema(rdd)
schemaRDD.registerTempTable("people")

当然，你也可以在Hive表里使用Spark SQL

刚才的table建好后，你就可以在上面跑各种SQL了:

In [4]:
avgAge = sqlCtx.sql(
    """SELECT
           home,
           AVG(age) AS mean
       FROM people
       GROUP BY home"""
)
avgAge

MapPartitionsRDD[16] at mapPartitions at SerDeUtil.scala:143

大家都知道在一般的数据表里执行SQL之后得到的还是一份表格式的数据，在Spark SQL执行完之后，你得到的依旧是一个SchemaRDD。

In [5]:
avgAge.collect()

[Row(home=u'Brussels', mean=33.5),
 Row(home=u'Mechelen', mean=53.0),
 Row(home=u'Leuven', mean=42.0)]

因为SchemaRDD也是一种RDD，所以你之前学到的所有RDD上的transform或者action等operation都可以用，**同时你可以用`row.fieldname`取出来某个field**，如下:

In [6]:
print(avgAge
      .map(lambda row: "Average age in {0} is {1} years"
                        .format(row.home, row.mean))
      .reduce(lambda x, y: x + "\n" + y))

Average age in Brussels is 33.5 years
Average age in Mechelen is 53.0 years
Average age in Leuven is 42.0 years


### <font color='red'>练习作业</font><br>
**回顾一下，我们可以在pandas里定义这样一个dataframe**
```
data = {
    'country': ['BE', 'BE', 'BE', 'NL', 'NL', 'NL'],
    'year': [1913, 1950, 2003, 1913, 1950, 2003],
    'gdp_per_capita': [4220, 5462, 21205, 4049, 5996, 21480]
}
frame = DataFrame(data)
```
**请大家在Spark中生成一个SchemaRDD，再做一个简单的SQL分析：求这3年各首都的平均GDP**

再次提醒一下：`sqlCtx.sql("...")`得到的结果本身也是一个`SchemaRDD`，所以你也可以把它注册成一张新的表，再上门做后续的操作！

### <font color='red'>练习作业</font><br>
**`sc.textFile`, `union` 和 `map`练习。 把提供的`names`数据集构建成一个SchemaRDD，包含`year`, `name`, `sex`和`births`，把它注册成一个表`"names"`**

**1. 求从1939到1945年美国总出生的人口**  

**2. 统计从1880到2014每一年叫`"Mary"`的宝宝出生数目，并用matplotlib绘制成图像**

**3. 统计从1880到2014每一年男孩和女孩的出生数，并绘制在一张图中。**

**4. 统计出来每一年出生的宝宝频次最高的前1000个名字，以及它们的占比，并用绘图的方式去展示出来**

## 更多知识
到此为止，应该Spark核心知识与Spark SQL知识你都有一些了解了，其实有更多的Spark SQL知识，比如:
* Complex Joins
* Subqueries
* Basic math functions
* User-defined functions (UDFs)
* ...

完整的文档可以看[这里](https://spark.apache.org/docs/2.1.0/sql-programming-guide.html#compatibility-with-apache-hive)